# 6.5.2

In [1]:
# %pip install gmaps
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV created
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Yellowknife,CA,2021-07-12 01:20:20,62.4560,-114.3525,73.92,49,75,3.00
1,1,Forestville,US,2021-07-12 01:20:20,38.8451,-76.8750,84.65,78,1,5.75
2,2,Kapaa,US,2021-07-12 01:16:15,22.0752,-159.3190,87.78,72,20,7.00
3,3,Buraydah,SA,2021-07-12 01:20:21,26.3260,43.9750,88.25,10,96,8.14
4,4,Saint-Philippe,RE,2021-07-12 01:20:21,-21.3585,55.7679,67.55,76,44,11.88


In [3]:
# check data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Config gmaps to use my Google API Key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the max temps
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temp
# Get the lat & long
locations = city_data_df[["Lat", "Lng"]]

# Get the max temp
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
city_data_df["Humidity"] = city_data_df["Humidity"].astype('float64')
humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
city_data_df["Cloudiness"] = city_data_df["Cloudiness"].astype('float64')
clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

# 6.5.3

In [10]:
# Ask the customer to add a min and max temp value

min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [11]:
# Filter the  dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df[(city_data_df["Max Temp"] <= max_temp) & \
                                  (city_data_df["Max Temp"] > min_temp)]
preferred_cities_df.head(10)
preferred_cities_df.count()

City_ID       222
City          222
Country       222
Date          222
Lat           222
Lng           222
Max Temp      222
Humidity      222
Cloudiness    222
Wind Speed    222
dtype: int64

# 6.5.4

In [44]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Forestville,US,84.65,38.8451,-76.8750,
2,Kapaa,US,87.78,22.0752,-159.3190,
3,Buraydah,SA,88.25,26.3260,43.9750,
5,Kathu,TH,82.62,7.9178,98.3332,
6,Bacolod,PH,84.34,10.6667,122.9500,
20,Brasileia,BR,79.36,-11.0161,-68.7481,
22,Chunhuhub,MX,83.16,19.5500,-88.6833,
28,Panama City,PA,83.05,8.9936,-79.5197,
30,Kirakira,SB,84.25,-10.4544,161.9205,
31,Kalmunai,LK,78.75,7.4167,81.8167,


In [59]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [60]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [62]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Forestville,US,84.65,38.8451,-76.8750,Quality Inn Near Joint Base Andrews-Washington...
2,Kapaa,US,87.78,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Buraydah,SA,88.25,26.3260,43.9750,فندق راحة السلام
5,Kathu,TH,82.62,7.9178,98.3332,Novotel Phuket Resort
6,Bacolod,PH,84.34,10.6667,122.9500,Planta Hotel & Residences
20,Brasileia,BR,79.36,-11.0161,-68.7481,Pousada Tobias
22,Chunhuhub,MX,83.16,19.5500,-88.6833,Chunhubub
28,Panama City,PA,83.05,8.9936,-79.5197,Hotel Riande Granada Urban
30,Kirakira,SB,84.25,-10.4544,161.9205,SanBiz Lodge
31,Kalmunai,LK,78.75,7.4167,81.8167,VS Villa


In [76]:
# Create an infobox
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temp for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))